In [3]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

In [3]:
# truncatedModelPath = "truncatedModel"

# truncatedTokenizer = AutoTokenizer.from_pretrained(truncatedModelPath)

fullMT5Name = "google/mt5-small"

#fullModel = AutoModelForSeq2SeqLM.from_pretrained(fullMT5Name)
fullTokenizer = AutoTokenizer.from_pretrained(fullMT5Name)

In [4]:
fullTokenizer = AutoTokenizer.from_pretrained("NewSP")

In [5]:
fullTokenizer

PreTrainedTokenizerFast(name_or_path='NewSP', vocab_size=10778, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42

In [8]:
fullTokenizer.convert_ids_to_tokens([i for i in range(250000, 250100)])

['▁<extra_id_99>',
 '▁<extra_id_98>',
 '▁<extra_id_97>',
 '▁<extra_id_96>',
 '▁<extra_id_95>',
 '▁<extra_id_94>',
 '▁<extra_id_93>',
 '▁<extra_id_92>',
 '▁<extra_id_91>',
 '▁<extra_id_90>',
 '▁<extra_id_89>',
 '▁<extra_id_88>',
 '▁<extra_id_87>',
 '▁<extra_id_86>',
 '▁<extra_id_85>',
 '▁<extra_id_84>',
 '▁<extra_id_83>',
 '▁<extra_id_82>',
 '▁<extra_id_81>',
 '▁<extra_id_80>',
 '▁<extra_id_79>',
 '▁<extra_id_78>',
 '▁<extra_id_77>',
 '▁<extra_id_76>',
 '▁<extra_id_75>',
 '▁<extra_id_74>',
 '▁<extra_id_73>',
 '▁<extra_id_72>',
 '▁<extra_id_71>',
 '▁<extra_id_70>',
 '▁<extra_id_69>',
 '▁<extra_id_68>',
 '▁<extra_id_67>',
 '▁<extra_id_66>',
 '▁<extra_id_65>',
 '▁<extra_id_64>',
 '▁<extra_id_63>',
 '▁<extra_id_62>',
 '▁<extra_id_61>',
 '▁<extra_id_60>',
 '▁<extra_id_59>',
 '▁<extra_id_58>',
 '▁<extra_id_57>',
 '▁<extra_id_56>',
 '▁<extra_id_55>',
 '▁<extra_id_54>',
 '▁<extra_id_53>',
 '▁<extra_id_52>',
 '▁<extra_id_51>',
 '▁<extra_id_50>',
 '▁<extra_id_49>',
 '▁<extra_id_48>',
 '▁<extra_id

In [7]:
keepTokens = truncatedTokenizer.convert_ids_to_tokens([i for i in range(259, truncatedTokenizer.vocab_size-100)])
keepIdInFull = [i for i in range(259)] + fullTokenizer.convert_tokens_to_ids(keepTokens) + [i for i in range(fullTokenizer.vocab_size-100, fullTokenizer.vocab_size)]


from copy import deepcopy
truncatedModel = deepcopy(fullModel)
truncatedModel.config.vocab_size=len(keepIdInFull)

In [8]:
from torch import nn

truncatedModel.encoder.embed_tokens = nn.Embedding(len(keepIdInFull), truncatedModel.config.hidden_size)
truncatedModel.encoder.embed_tokens.data = fullModel.encoder.embed_tokens.weight[keepIdInFull].detach().numpy()

truncatedModel.shared = truncatedModel.encoder.embed_tokens

truncatedModel.decoder.embed_tokens = nn.Embedding(len(keepIdInFull), truncatedModel.config.hidden_size)
truncatedModel.decoder.embed_tokens.data = fullModel.decoder.embed_tokens.weight[keepIdInFull].detach().numpy()

truncatedModel.shared = truncatedModel.decoder.embed_tokens

truncatedModel.lm_head = nn.Linear(truncatedModel.config.hidden_size, truncatedModel.config.vocab_size, bias=False)
truncatedModel.lm_head.weight.data = fullModel.lm_head.weight.data[keepIdInFull]

In [ ]:
print(truncatedModel)

In [10]:
truncatedModel.save_pretrained(truncatedModelPath)